In [1]:
import numpy as np
import torch
from networks.hyper import HyperDip
from hypnettorch.hnets import StructuredHMLP
from hypnettorch.hnets.structured_hmlp_examples import resnet_chunking
from hypnettorch.hnets.chunked_mlp_hnet import ChunkedHMLP

In [2]:
net = HyperDip(3, 1,
                num_channels_down = [128, 128, 128, 128, 128],
                num_channels_up   = [128, 128, 128, 128, 128],
                num_channels_skip = [16, 16, 16, 16, 16],
                upsample_mode='bilinear',
                need_sigmoid=True, need_bias=True, pad='reflection', act_fun='LeakyReLU').cuda()

In [3]:
net._is_properly_setup()

In [4]:
hnet = ChunkedHMLP(net.hyper_shapes_learned, 16384, layers=(128,128,128), use_batch_norm=True).cuda()
# hnet.apply_chunked_hyperfan_init()

Created Chunked MLP Hypernet with 144 chunk(s) of size 16384.
Hypernetwork with 2150664 weights and 2352193 outputs (compression ratio: 0.91).
The network consists of 2150656 unconditional weights (2150656 internally maintained) and 8 conditional weights (8 internally maintained).


In [5]:
import time
x_ = torch.rand(9, 8).cuda()
start = time.perf_counter()
weights = hnet.forward(cond_input=x_)
weight_gen =  time.perf_counter()
print("WEIGHT GENERATION: ", weight_gen - start)
x = torch.rand(9, 3, 300, 300).cuda()
with torch.no_grad():
    y1 = net.forward(x, weights=weights[0])
end = time.perf_counter()
print("NET FORWARD: ", end - weight_gen)



WEIGHT GENERATION:  0.43922084599762456
76 38 38
NET FORWARD:  0.15073687700351002


/home/doseok/miniconda3/envs/225BProject/lib/python3.7/site-packages/torch/nn/functional.py:3635: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)
